In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="ticks", color_codes=True)
from mpl_toolkits.basemap import Basemap
from statsmodels.formula.api import ols

In [ ]:
train=pd.read_csv('../input/train.csv',nrows=5000000)
test=pd.read_csv('../input/test.csv')

#### Ok time to start lets take a look 

In [ ]:
train.head()

#### First lets check for null valuse and drop them

In [ ]:
train.isna().sum()

In [ ]:
train=train.dropna()

### Analysis and PreProcessing

#### My first 3 question to ask after looking at this features are :
#### is passenger count affect the fare?
#### ofcourse the distance matter but how?
#### does the time of the ride affect ? was it weekend or it differ at night more than other ?

### Lets start with passenger count

In [ ]:
train['passenger_count'].unique()

In [ ]:
test['passenger_count'].unique()

#### well i havn't heard of taxi that take more than 6 person actually so there is something wrong here for sure , so i will drop passenger>6  and ofcourse 0 count i willn't assume anything here this will mislead the model 

In [ ]:
train=train[(train['passenger_count']<=6)&(train['passenger_count']>0)]

In [ ]:
len(train)

#### lets take a look at fare amount variable

In [ ]:
train['fare_amount'].describe()

### 3 things to notice here 
#### first the standard dev is high which means there is a huge difference between our points also might be alot of outliers
#### second the min value is negtavie which can't be real so remove it
#### third the max value is 1273  , who will pay this amout for taxi trip XD

In [ ]:
#remove negative faer value
train=train[train['fare_amount']>0]

In [ ]:
sns.boxplot(train['fare_amount'])

In [ ]:
train[train['fare_amount']>600]

#### ok so this points is annoying outliers i will remove them 

In [ ]:
train=train[train['fare_amount']<600]

#### from the graph most of fare values falls between 0~100 but there is a big number of data that falls >100 

In [ ]:
plot=sns.distplot(train['fare_amount'])

#### time to see thre realation between passenger count and fare 

In [ ]:
sns.catplot(x='passenger_count',y='fare_amount',data=train)

#### it seems that passenger count with 1 paid the biggest fares and the more the passengers we see less fares

#### just to make sure i will make simple hypothesis test to check if this means differ because of the number of passengers

In [ ]:
model=ols('fare_amount~passenger_count',data=train).fit()
model.summary()

#### ok this is low probability value and hight f score so everything is okay this features is important

#### Time for distance features and make use of this longitudes and latitudes

In [ ]:
train[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].describe()

#### again numbers tells everything 
#### longitudes should be (-180~180) which is not the case here same for latitudes (-90~90)
#### the deviation in the latitudes values is huge so there is a problem caouse this number should all fall within newyork range which is between((40~43),(-72,-75))

#### first i will deal with >180 and >90 points and the rest i will explore more to see if i'm going to remove or let them based on distance values

In [ ]:
train=train[(train['pickup_longitude']>-180) & (train['pickup_longitude']<180)]

In [ ]:
train=train[(train['pickup_latitude']>-90) & (train['pickup_latitude']<90)]

In [ ]:
train=train[(train['dropoff_longitude']>-180) & (train['dropoff_longitude']<180)]

In [ ]:
train=train[(train['dropoff_latitude']>-90) & (train['dropoff_latitude']<90)]

In [ ]:
train[['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude']].describe()

#### visualize this point on map gives us better information about this pick up places and dropoff
#### something like tableau would be very good here but i decided to use BaseMap libirairy in python 

In [ ]:
fares=train.fare_amount.values

In [ ]:
#draw pick up points
fig = plt.figure(figsize=(30, 10))
m = Basemap(projection='cyl',llcrnrlat=40,urcrnrlat=42,\
            llcrnrlon=-75, urcrnrlon=-72, resolution='h', area_thresh=70, lat_0=40.78, lon_0=-73.96)

lons = train['pickup_longitude'].values
lats = train['pickup_latitude'].values
x,y=m(lons,lats)
### there is a problem with basemap version here that causes drawmapboundry to color everything without this bug u could color the sea with anyother color
#m.drawmapboundary(fill_color='lightblue')
m.fillcontinents(color='lightgrey',zorder=0)
m.drawstates(color='black')
z=m.scatter(x,y,c=fares,cmap='Reds')
cbar=m.colorbar(z)
cbar.set_label('fares')

In [ ]:
#draw dropoff points
fig = plt.figure(figsize=(30, 10))
m = Basemap(projection='cyl',llcrnrlat=40,urcrnrlat=42,\
            llcrnrlon=-75, urcrnrlon=-72, resolution='h', area_thresh=70, lat_0=40.78, lon_0=-73.96)

lons = train['dropoff_longitude'].values
lats = train['dropoff_latitude'].values
x,y=m(lons,lats)
### there is a problem with basemap version here that causes drawmapboundry to color everything without this bug u could color the sea with anyother color
#m.drawmapboundary(fill_color='lightblue')
m.fillcontinents(color='lightgrey',zorder=0)
m.drawstates(color='black')
z=m.scatter(x,y,c=fares,cmap='Reds')
cbar=m.colorbar(z)
cbar.set_label('fares')

#### There is somepoints in the sea here but i will choose to let them cause whatever distance metric i use it will not assume anythin about the roads the drive take so i see no problem from letting them 

In [ ]:
#draw pick up for test data
fig = plt.figure(figsize=(30, 10))
m = Basemap(projection='cyl',llcrnrlat=40,urcrnrlat=42,\
            llcrnrlon=-75, urcrnrlon=-72, resolution='h', area_thresh=70, lat_0=40.78, lon_0=-73.96)

lons = test['pickup_longitude'].values
lats = test['pickup_latitude'].values
x,y=m(lons,lats)
### there is a problem with basemap version here that causes drawmapboundry to color everything without this bug u could color the sea with anyother color
#m.drawmapboundary(fill_color='lightblue')
m.fillcontinents(color='lightgrey',zorder=0)
m.drawstates(color='black')
m.scatter(x,y)

In [ ]:
#draw drofoff for test data
fig = plt.figure(figsize=(30, 10))
m = Basemap(projection='cyl',llcrnrlat=40,urcrnrlat=42,\
            llcrnrlon=-75, urcrnrlon=-72, resolution='h', area_thresh=70, lat_0=40.78, lon_0=-73.96)

lons = test['dropoff_longitude'].values
lats = test['dropoff_latitude'].values
x,y=m(lons,lats)
### there is a problem with basemap version here that causes drawmapboundry to color everything without this bug u could color the sea with anyother color
#m.drawmapboundary(fill_color='lightblue')
m.fillcontinents(color='lightgrey',zorder=0)
m.drawstates(color='black')
m.scatter(x,y)

#### OK lets calculate distance i will use haversine distance according to this [link here](https://www.movable-type.co.uk/scripts/latlong.html)
#### i'm also using this [post](https://stackoverflow.com/questions/25767596/vectorised-haversine-formula-with-a-pandas-dataframe) to select the best performance code to get the distance because we deal with large data 

In [ ]:
train['harv_distance']=6371 * 2 * np.arcsin(np.sqrt(np.sin((np.radians(train['dropoff_latitude']) -np.radians(train['pickup_latitude']))/2)**2 + np.cos(np.radians(train['pickup_latitude'])) * np.cos(np.radians(train['dropoff_latitude'])) * np.sin((np.radians(train['dropoff_longitude']) - np.radians(train['pickup_longitude']))/2)**2))

In [ ]:
train.harv_distance.describe()

In [ ]:
train.harv_distance.max()

#### So there is a 0 distance and 12K KM WOW , who would ride 12K KM  in taxi i will take a deep look

In [ ]:
train.sort_values('harv_distance', ascending=False)

In [ ]:
sns.scatterplot(x='harv_distance',y='fare_amount',data=train)

#### from the above graphs and data somethings to notice here 

#### first there are somepoints that have 0 pick up long and lat and 0 dropoff long and lat which i think missing data and need to ber removed 

#### second there are some pick up points that is the same as the drop off points and again i will not  assume that it was round trip cause 0 distance with high fare value is going to make noise in the model no other information to tell it that it is round trip so i will also drop them

In [ ]:
train=train[(train['pickup_longitude']!=train['dropoff_longitude']) & (train['pickup_latitude']!=train['dropoff_latitude'])]

#### ok another thing here is that there is a huge distance valuse so i want to look closer to this points that make this huge value

In [ ]:
train[(train['harv_distance']>500)&(train['harv_distance']<6000)].sort_values(['harv_distance'],ascending=True)

In [ ]:
fig = plt.figure(figsize=(30, 10))
m = Basemap(projection='cyl',llcrnrlat=7,urcrnrlat=90,\
            llcrnrlon=-90, urcrnrlon=-4, resolution='h', area_thresh=50)

lons1 = -73.99503
lats1 = 40.744945
x1,y1=m(lons1,lats1)

lons2 = -7.98664
lats2 = 40.729937
x2,y2=m(lons2,lats2)
### there is a problem with basemap version here that causes drawmapboundry to color everything without this bug u could color the sea with anyother color
#m.drawmapboundary(fill_color='lightblue')
m.fillcontinents(color='lightgrey',zorder=0)
m.drawstates(color='black')
m.scatter(x1,y1,color='blue',marker='x')
m.scatter(x2,y2,color='red',marker='o')
plt.arrow(x1,y1,x2-x1,y2-y1,color='black')

In [ ]:
fig = plt.figure(figsize=(30, 10))
m = Basemap(projection='cyl',llcrnrlat=40,urcrnrlat=45,\
            llcrnrlon=-80, urcrnrlon=-70, resolution='h', area_thresh=50)

lons1 = -73.977917
lats1 = 40.752368
x1,y1=m(lons1,lats1)

lons2 = -79.090594
lats2 = 43.178567
x2,y2=m(lons2,lats2)
### there is a problem with basemap version here that causes drawmapboundry to color everything without this bug u could color the sea with anyother color
#m.drawmapboundary(fill_color='lightblue')
m.fillcontinents(color='lightgrey',zorder=0)
m.drawstates(color='black')
m.scatter(x1,y1,color='blue',marker='x')
m.scatter(x2,y2,color='red',marker='o')

#### So i was thinking of finding a way to adjust this distances like find formula that calculates it but i found that this points is so missleading some of them cross the sea LOL and other is far far away from newyork (this is a flight not taxi trip) so i will choose to limit my data point to the area around Newyork only and drop every thing else

In [ ]:
train=train[(train['pickup_latitude']>=40) & (train['pickup_latitude']<=42)&\
            (train['dropoff_latitude']>=40) & (train['dropoff_latitude']<=42)&\
            (train['pickup_longitude']>-75) & (train['pickup_longitude']<=-72)&\
           (train['dropoff_longitude']>-75) & (train['dropoff_longitude']<=-72)]

In [ ]:
train['harv_distance']=6371 * 2 * np.arcsin(np.sqrt(np.sin((np.radians(train['dropoff_latitude']) -np.radians(train['pickup_latitude']))/2)**2 + np.cos(np.radians(train['pickup_latitude'])) * np.cos(np.radians(train['dropoff_latitude'])) * np.sin((np.radians(train['dropoff_longitude']) - np.radians(train['pickup_longitude']))/2)**2))

In [ ]:
sns.scatterplot(x='harv_distance',y='fare_amount',data=train)

#### that's better here there is some  linear relation between the distance and fare amount 

#### Also i can see here that there are some trips with short distance that have high fare amount and long trips with low fare amount, so there are some other features that affect our fare variable might be the time or something else alot of assumption to say here but i will stick with what i got 

In [ ]:
train.harv_distance.describe()

#### i want to look to the relation between passenger count and distance with fare amount might give us better insights

In [ ]:
groups=train.groupby('passenger_count')

In [ ]:
g=sns.FacetGrid(train,col='passenger_count')
g.map(plt.scatter,'harv_distance','fare_amount')

#### away from the 1 passsenger that have more varying relation , we see with high passenger count the relation tends to be less noisy for passanger count =6 we see they tend to take long distance with lets say normal fare amounts , and this is actually strange a little which make me think that there might be some places with fixed fare amount that have high distance and also large groups go there so i will make a guess that it is airport maybe , beaches !! 

#### actually i wanna look at the places that people got to more than other that might give us better information

In [ ]:
dup = train[train.duplicated(subset=['dropoff_longitude','dropoff_latitude'], keep=False)]

In [ ]:
repeated=dup.groupby(['dropoff_longitude','dropoff_latitude'])

In [ ]:
df=pd.DataFrame()
for g in repeated.groups:
    df=df.append({'long': g[0], 'lat': g[1], 'avg_fare': repeated.get_group(g)['fare_amount'].mean(),'avg_distance':repeated.get_group(g)['harv_distance'].mean(),'counts':repeated.get_group(g).count()['key']}, ignore_index=True)

In [ ]:
fig = plt.figure(figsize=(30, 10))
m = Basemap(projection='cyl',llcrnrlat=40,urcrnrlat=42,\
            llcrnrlon=-75, urcrnrlon=-72, resolution='h', area_thresh=70, lat_0=40.78, lon_0=-73.96)

counts=df.counts.values
lons=df.long.values
lans=df.lat.values
fares=df.avg_fare.values
x,y=m(lons,lans)
### there is a problem with basemap version here that causes drawmapboundry to color everything without this bug u could color the sea with anyother color
#m.drawmapboundary(fill_color='lightblue')
m.fillcontinents(color='lightgrey',zorder=0)
m.drawstates(color='black')
z=m.scatter(x,y,s=counts,c=fares,cmap='Reds')
cbar=m.colorbar(z)
cbar.set_label('fares')

#### this graph above shows the relation between repeated drop off places and fares we see that there is a point up there have frequent trips thany any other place but have low fair while some points with high fare amount have low frequent trips 

#### lets take deep look 

In [ ]:
sns.pairplot(df[['avg_fare','avg_distance','counts']])

#### Here we can see the total relation between this variables and that the assumption that there is some places that have high range fare values not fixed 

In [ ]:
df[df['counts']==df['counts'].max()]

In [ ]:
train[(train['dropoff_longitude']==-74.1771926879883)&(train['dropoff_latitude']==40.6949462890625)]

#### i googled this coordinates to check what are this places most frequent and highest fare i found most frequent one was place near FourMile Brook
#### and hight fares  was NewYork airports

#### what about pick up location 
#### Lets do the same thing and see what locations have most freqeuent trips and highest fares

In [ ]:
dup = train[train.duplicated(subset=['pickup_longitude','pickup_latitude'], keep=False)]
repeated=dup.groupby(['pickup_longitude','pickup_latitude'])
dt=pd.DataFrame()
for g in repeated.groups:
    dt=dt.append({'long': g[0], 'lat': g[1], 'avg_fare': repeated.get_group(g)['fare_amount'].mean(),'avg_distance':repeated.get_group(g)['harv_distance'].mean(),'counts':repeated.get_group(g).count()['key']}, ignore_index=True)

In [ ]:
fig = plt.figure(figsize=(30, 10))
m = Basemap(projection='cyl',llcrnrlat=40,urcrnrlat=42,\
            llcrnrlon=-75, urcrnrlon=-72, resolution='h', area_thresh=70, lat_0=40.78, lon_0=-73.96)

counts=dt.counts.values
lons=dt.long.values
lans=dt.lat.values
fares=dt.avg_fare.values
x,y=m(lons,lans)
### there is a problem with basemap version here that causes drawmapboundry to color everything without this bug u could color the sea with anyother color
#m.drawmapboundary(fill_color='lightblue')
m.fillcontinents(color='lightgrey',zorder=0)
m.drawstates(color='black')
z=m.scatter(x,y,s=counts,c=fares,cmap='Reds')
cbar=m.colorbar(z)
cbar.set_label('fares')

In [ ]:
sns.pairplot(dt[['avg_fare','avg_distance','counts']])

In [ ]:
dt.sort_values('counts',ascending=False).head()

In [ ]:
dt.sort_values('avg_fare',ascending=False).head()

#### Again after googling this points airports have highest fares amount and also there are big number of trips around LaGurdia airport and the area around East River 

In [ ]:
#### Now lets look at the datetime column

train['pickup_datetime']=pd.to_datetime(train['pickup_datetime'],infer_datetime_format=True)

train['year']=train.pickup_datetime.apply(lambda x:x.year)

train['day']=train.pickup_datetime.apply(lambda x:x.weekday())

train['month']=train.pickup_datetime.apply(lambda x:x.month)

train['hour']=train.pickup_datetime.apply(lambda x:x.hour)

In [ ]:
train.head()

So i want to look at the mean fare value at each hour for each day and see if there is a relation

In [ ]:
groups=train.groupby(['day','hour'])
data=groups.mean()['fare_amount'].reset_index()


In [ ]:
g=sns.FacetGrid(data,col='day',col_wrap=4)
g.map(sns.pointplot,'hour','fare_amount')

#### So here according to the mean value we can see that between 3 Am to 7 AM we have big fare values and because we calculate the mean that means that low number of trips we will see that in a minute and after 12PM more trips and normal fare values 

#### another thing to notice here is that the day is not affecting the fare values we see the same pattern happend each day

In [ ]:
sns.distplot(train['hour'],kde=False)

#### as i said between 3Am and 7AM more trips 

### Lets check the day

In [ ]:
sns.catplot(x='day',y='fare_amount',data=train)

In [ ]:
pd.crosstab(train['fare_amount'],train['day']).plot(kind='kde',xlim=[-1000,1000])

#### cant see big difference here even with weekend days 

### What about the year 

In [ ]:
groups=train.groupby(['year','hour'])
data=groups.mean()['fare_amount'].reset_index()
g=sns.FacetGrid(data,col='year',col_wrap=4)
g.map(sns.pointplot,'hour','fare_amount')

#### Seems like year really matter the fare amount between 2012~2015 is larger than rest of years might be Gasoline prices 

### Now prepare the data for the model 
#### The features i will select is (distance-hour-year-passenger Count)

In [ ]:
test['pickup_datetime']=pd.to_datetime(test['pickup_datetime'],infer_datetime_format=True)
test['year']=test.pickup_datetime.apply(lambda x:x.year)
test['day']=test.pickup_datetime.apply(lambda x:x.weekday())
test['hour']=test.pickup_datetime.apply(lambda x:x.hour)
test['harv_distance']=6371 * 2 * np.arcsin(np.sqrt(np.sin((np.radians(test['dropoff_latitude']) -np.radians(test['pickup_latitude']))/2)**2 + np.cos(np.radians(test['pickup_latitude'])) * np.cos(np.radians(test['dropoff_latitude'])) * np.sin((np.radians(test['dropoff_longitude']) - np.radians(test['pickup_longitude']))/2)**2))

In [ ]:
test.head()

#### so after all the analysis we did we know that location is important the distance not just say everything if i could pass the locations to the model i think i it's going to do better(actually i tested that and i rmse dropped 1 unit after adding it)
#### so to pass tha locations we can't pass the longitude and latitude so to we need variables that represent 3 dim data 
#### the formula is 
#### X=cos(lat)*cos(long)
#### y=cos(lat)*sin(long)
#### z=sin(lat)

In [ ]:
train['xpick']=np.cos(train['pickup_latitude'])*np.cos(train['pickup_longitude'])
train['ypick']=np.cos(train['pickup_latitude'])*np.sin(train['pickup_longitude'])
train['zpick']=np.sin(train['pickup_latitude'])
train['xdrop']=np.cos(train['dropoff_latitude'])*np.cos(train['dropoff_longitude'])
train['ydrop']=np.cos(train['dropoff_latitude'])*np.cos(train['dropoff_longitude'])
train['zdrop']=np.sin(train['dropoff_latitude'])
test['xpick']=np.cos(test['pickup_latitude'])*np.cos(test['pickup_longitude'])
test['ypick']=np.cos(test['pickup_latitude'])*np.sin(test['pickup_longitude'])
test['zpick']=np.sin(test['pickup_latitude'])
test['xdrop']=np.cos(test['dropoff_latitude'])*np.cos(test['dropoff_longitude'])
test['ydrop']=np.cos(test['dropoff_latitude'])*np.cos(test['dropoff_longitude'])
test['zdrop']=np.sin(test['dropoff_latitude'])

In [ ]:
features = ['year', 'hour', 'harv_distance','day', 'passenger_count','xpick','ypick','zpick','xdrop','ydrop','zdrop']
X = train[features].values
y = train['fare_amount'].values

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### i'm going to use random forest regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create the random forest
random_forest = RandomForestRegressor(n_estimators = 20, max_depth = 20, 
                                      max_features = None, oob_score = True, 
                                      bootstrap = True, verbose = 1, n_jobs = -1)

# Train on data
random_forest.fit(X_train, y_train)

In [ ]:
y_pred = random_forest.predict(X_test)
fig, axs = plt.subplots(1,1,figsize=(10,4))
axs.scatter(y_test, y_pred)
print("Mean Squared Error: %.4f"
      % np.sqrt(np.mean((y_pred - y_test) ** 2)))

In [ ]:
XTEST = test[features].values
filename = './output/rf_model'

y_pred_final = random_forest.predict(XTEST)

submission = pd.DataFrame(
    {'key': test.key, 'fare_amount': y_pred_final},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission1.csv', index = False)